In [ ]:
use database demo_map;
use schema addresses;

In [ ]:
# We can also use Snowpark for our analyses!
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
session = get_active_session()
import snowflake.snowpark.functions as F

In [ ]:
addr_query = "SELECT  ADDR_STRING FROM demo_map.addresses.ADDRESSES_TO_PARSE WHERE ADDR_STRING IS NOT NULL"
addr_data = session.sql(addr_query)
addr_data.show()

In [ ]:
 
addr_data = session.sql("SELECT  ADDR_STRING FROM demo_map.addresses.ADDRESSES_TO_PARSE WHERE ADDR_STRING IS NOT NULL")
addr_data.show()

In [ ]:
current_df  = session.table("demo_map.addresses.ADDRESSES_TO_PARSE")
current_df.show()

In [ ]:
import re
import pandas as pd

# Define the regex pattern for address parsing
def parse_address(address):
    pattern = r'(\d+[A-Za-z0-9]*)\s+([A-Za-z]+(?:\s[A-Za-z]+)*)\s+(St|Ave|Blvd|Rd|Ln|Dr|Ct|Pkwy|Road)\s*,?\s*([A-Za-z\s]+),\s*([A-Za-z]{2})\s*(\d{5})'
    
    match = re.match(pattern, address)
    
    if match:
        house_number = match.group(1)
        street_name = match.group(2)
        street_type = match.group(3)
        city = match.group(4).strip()
        state = match.group(5)
        zip_code = match.group(6)
        return house_number, street_name, street_type, city, state, zip_code
    else:
        return None, None, None, None, None, None

# Retrieve data from the Snowflake table
#addresses_df = session.sql("SELECT   ADDR_STRING FROM demo_map.addresses.ADDRESSES_TO_PARSE WHERE ADDR_STRING IS NOT NULL")
addresses_df = session.sql("SELECT ROW_ID, ADDR_STRING FROM demo_map.addresses.ADDRESSES_TO_PARSE WHERE ADDR_STRING IS NOT NULL")

# Collect Snowpark DataFrame into Pandas DataFrame
addresses_df = addresses_df.to_pandas()

 

# Parse the address for each row and store the results in separate columns
addresses_df[['NUMBER', 'STREET', 'STREET_TYPE', 'CITY', 'STATE', 'ZIP']] = addresses_df['ADDR_STRING'].apply(
    lambda x: pd.Series(parse_address(x))
)


# print(addresses_df.head()) 
#print(addresses_df)
table_name = "ADDRESSES_PARSED"

# commented out below, it was for initial table creation
#session.write_pandas(addresses_df, table_name, auto_create_table=True, overwrite=True)

# for ongoing purposes we want to append, so will change overwrite=False
session.write_pandas(addresses_df, table_name, auto_create_table=True, overwrite=False)
 
 
 



In [ ]:
 
get_active_session()
